<a href="https://colab.research.google.com/github/Arockiajerin/CT-Lung-cancer-prediction-using-CNN/blob/main/ctlungcancerprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!unrar x "/content/drive/MyDrive/lung_image_sets1.rar" "/content/extract/"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/lung_image_sets1.rar

Creating    /content/extract                                          OK
Creating    /content/extract/lung_image_sets1                         OK
Creating    /content/extract/lung_image_sets1/lung_aca                OK
Extracting  /content/extract/lung_image_sets1/lung_aca/lungaca1.jpeg       0%  OK 
Extracting  /content/extract/lung_image_sets1/lung_aca/lungaca10.jpeg       0%  OK 
Extracting  /content/extract/lung_image_sets1/lung_aca/lungaca100.jpeg       0%  OK 
Extracting  /content/extract/lung_image_sets1/lung_aca/lungaca1000.jpeg       0%  OK 
Extracting  /content/extract/lung_image_sets1/lung_aca/lungaca1001.jpeg       0%  OK 
Extracting  /content/extract/lung_image_sets1/lung_aca/lungaca101.jpeg       0%  OK 
Extracting  /content/extract/lung_image_sets1/lung_aca/lungaca102.jpeg       

In [ ]:
import os
import cv2
import numpy as np

# Define correct dataset path
DATASET_PATH = "/content/extract/lung_image_sets1"
IMG_SIZE = (224, 224)

def load_and_preprocess_images(directory):
    images, labels = [], []

    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)

        if os.path.isdir(class_path):  # Ensure it's a folder
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)

                # Debugging: Check if image exists
                if not os.path.exists(img_path):
                    print(f"File not found: {img_path}")
                    continue

                # Read and preprocess image
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Error loading: {img_path}")
                    continue

                img = cv2.resize(img, IMG_SIZE)
                img = img / 255.0  # Normalize

                images.append(img)
                labels.append(class_name)

    return np.array(images), np.array(labels)

X, y = load_and_preprocess_images(DATASET_PATH)

# Reshape for CNN
X = X.reshape(-1, IMG_SIZE[0], IMG_SIZE[1], 1)

print(f"Loaded {X.shape[0]} images, each of shape {X.shape[1:]}")


Loaded 3003 images, each of shape (224, 224, 1)


In [ ]:
from sklearn.model_selection import train_test_split

# Split into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training data: {X_train.shape[0]} images")
print(f"Testing data: {X_test.shape[0]} images")


Training data: 2402 images
Testing data: 601 images


In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Convert labels to numbers
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Convert to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print("Labels converted to categorical format!")


Labels converted to categorical format!


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Prevent overfitting
    Dense(len(set(y_train.argmax(axis=1))), activation='softmax')  # Multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 222, 222, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,168,771 (42.61 MB)

 Trainable params: 11,168,771 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the CNN model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32
)


Epoch 1/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 306s 4s/step - accuracy: 0.9610 - loss: 0.1031 - val_accuracy: 0.8319 - val_loss: 0.4605
Epoch 2/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 279s 4s/step - accuracy: 0.9732 - loss: 0.0887 - val_accuracy: 0.8419 - val_loss: 0.5738
Epoch 3/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 344s 4s/step - accuracy: 0.9777 - loss: 0.0680 - val_accuracy: 0.8037 - val_loss: 0.6756
Epoch 4/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 286s 4s/step - accuracy: 0.9787 - loss: 0.0589 - val_accuracy: 0.6722 - val_loss: 1.7706
Epoch 5/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 321s 4s/step - accuracy: 0.9001 - loss: 0.3578 - val_accuracy: 0.8220 - val_loss: 0.5303
Epoch 6/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 317s 4s/step - accuracy: 0.9863 - loss: 0.0544 - val_accuracy: 0.7920 - val_loss: 0.6998
Epoch 7/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 280s 4s/step - accuracy: 0.9846 - loss: 0.0484 - val_accuracy: 0.7970 - val_loss: 0.7144
Epoch 8/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 323s 4s/step - accuracy: 0.9913 - loss: 0.0468 - val_accuracy: 0.8203 - v

In [ ]:
# Evaluate the model on test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")

19/19 ━━━━━━━━━━━━━━━━━━━━ 19s 972ms/step - accuracy: 0.8244 - loss: 0.6509
Test Accuracy: 83.53%


In [ ]:
# Save the model as an H5 file
model.save("/content/lung_cancer_cnn.h5")

print("Model saved successfully!")


Model saved successfully!


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("/content/lung_cancer_cnn.h5")

print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
import cv2
import numpy as np

def predict_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224, 224))  # Resize to match CNN input size
    img = img / 255.0  # Normalize
    img = img.reshape(1, 224, 224, 1)  # Reshape for model input

    # Make prediction
    prediction = model.predict(img)
    class_index = np.argmax(prediction)  # Get predicted class
    confidence = np.max(prediction) * 100  # Get confidence score (percentage)
    class_label = label_encoder.inverse_transform([class_index])[0]

    # Determine Cancer or No Cancer
    if "lung_n" in class_label.lower():  # Assuming "lung_n" means normal (no cancer)
        status = "No Cancer"
    else:
        status = "Cancer Detected"

    print(f"Predicted Class: {class_label} ({confidence:.2f}% confidence) - {status}")

# Test with a sample image
predict_image("/content/extract/lung_image_sets1/lung_aca/lungaca1001.jpeg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
Predicted Class: lung_aca (100.00% confidence) - Cancer Detected


In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model("/content/lung_cancer_cnn.h5")

# Define class labels manually (ensure this matches your training order)
class_names = ["lung_n", "lung_scc", "lung_aca"]  # Modify if needed

def predict_image_gradio(image):
    # Convert to grayscale
    image = image.convert("L")
    img = np.array(image)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = img.reshape(1, 224, 224, 1)

    # Predict
    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    confidence = np.max(prediction) * 100
    class_label = label_encoder.inverse_transform([class_index])[0]

    # Determine status
    if "lung_n" in class_label.lower():
        status = "🟢 No Cancer"
    else:
        status = "🔴 Cancer Detected"

    return f"Class: {class_label}\nConfidence: {confidence:.2f}%\nStatus: {status}"

# Gradio interface
interface = gr.Interface(
    fn=predict_image_gradio,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Lung Cancer Detection from CT Images",
    description="Upload a CT scan image to predict whether it indicates lung cancer.",
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d846dd51973b720b4e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
